In [1]:
from platform import python_version

print(python_version())

In [2]:
%matplotlib inline
import time
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models


In [3]:
np.random.seed(0)

In [4]:
import os

In [5]:
# len('chili-dog')

In [6]:
def read_images(path="./hw4/train_kaggle"):
    imgs = os.listdir(path)
    X, y = [], []
    for i in imgs:
        X.append(i)
#         'frankfurter', 'chili-dog' or 'hotdog'
        y.append(1 if len(i) > 6 and 
                 (i[:6]=='hotdog' or i[:11] == 'frankfurter' or i[:9] == 'chili-dog')
                 else 0)
    return np.array(X), np.array(y, dtype=np.int)

In [7]:
X, y = read_images()
# X = X[:60]
# y = y[:60]

In [8]:
torch.cuda.is_available(), np.mean(y)

(True, 0.3137084510102107)

In [9]:
resnet = models.resnet152(pretrained=True)

In [10]:
for param in resnet.parameters():
    param.requires_grad = False

In [11]:
print(resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
# resnet.fc.out_features = 2

In [13]:
resnet.fc = nn.Sequential(
   nn.Linear(2048, 128),
   nn.ReLU(inplace=True),
   nn.Dropout(0.4),
   nn.Linear(128, 2))
#     nn.Linear(fc_inputs, 256),
#     nn.ReLU(),
#     nn.Dropout(0.4),
#     nn.Linear(fc_inputs, 2), 
#     nn.Softmax() # For using NLLLoss()
# )
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
for param in resnet.fc.parameters():
    param.requires_grad = True

In [15]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        print('kjdbkvj')
        nn.init.kaiming_normal(m.weight.data)
        
resnet.fc.apply(weights_init);

kjdbkvj
kjdbkvj


/home/nastya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  """


In [22]:
params = {
    'batch_size': 24,
    'shuffle': True,
    'num_workers': 8
}


In [23]:
ds_trans = transforms.Compose([transforms.Scale(224),
                               transforms.CenterCrop(224),
                               transforms.RandomHorizontalFlip(),
                               transforms.RandomAffine(8, shear=5, scale=(0.85,1.15)),
                               transforms.ToTensor()])



/home/nastya/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [24]:
from PIL import Image

class MyDataset(Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, list_IDs, labels, transform, datadir='./hw4/train_kaggle'):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs
        self.transform = transform
        self.datadir = datadir

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]       
        img = Image.open(self.datadir + '/' + ID)
        X = self.transform(img)
        y = self.labels[index]
        return X, y

In [25]:
np.sum(y)/y.shape[0]
torch.cuda.is_available()

True

In [26]:
# fc_inputs = resnet.fc.in_features
 
# resnet.fc = nn.Sequential(
# #    nn.Linear(2048, 128),
# #    nn.ReLU(inplace=True),
# #    nn.Dropout(0.4),
#    nn.Linear(2048, 2))
# #     nn.Linear(fc_inputs, 256),
# #     nn.ReLU(),
# #     nn.Dropout(0.4),
# #     nn.Linear(fc_inputs, 2), 
# #     nn.Softmax() # For using NLLLoss()
# # )


In [27]:
resnet = resnet.cuda()
criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.3137, 1 - 0.3137]).cuda())
optimizer = torch.optim.Adam(resnet.parameters(),lr=0.001, weight_decay=1e-6)
# optimizer = torch.optim.SGD(resnet.parameters(),lr=0.001, momentum=0.9, nesterov=True, weight_decay=1e-6)

In [ ]:
import time
training_set = MyDataset(X, y, ds_trans)
training_generator = DataLoader(training_set, **params)
epochs = 150
losses = []
accs = []
for epoch in range(epochs):
    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch+1, epochs))
     
    # Set to training mode
    resnet.train()
     
    # Loss and Accuracy within the epoch
    train_loss = 0.0
    train_acc = 0.0
     
    valid_loss = 0.0
    valid_acc = 0.0
 
    for i, (inputs, labels) in enumerate(training_generator):
 
        inputs = inputs.cuda()
        labels = labels.cuda()
         
        # Clean existing gradients
        optimizer.zero_grad()
         
        # Forward pass - compute outputs on input data using the model
        outputs = resnet(inputs)
#         print(outputs)
#         print(torch.max(labels, 1)[1])
        # Compute loss
        loss = criterion(outputs.float(), labels)
         
        # Backpropagate the gradients
        loss.backward()
         
        # Update the parameters
        optimizer.step()
         
        # Compute the total loss for the batch and add it to train_loss
        train_loss += loss.item() * inputs.size(0)
         
        # Compute the accuracy
        ret, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(labels.data.view_as(predictions))
         
        # Convert correct_counts to float and then compute the mean
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
         
        # Compute total accuracy in the whole batch and add to train_acc
        train_acc += acc.item() * inputs.size(0)
        losses.append(loss.item()) 
        accs.append(acc.item())
        print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(
            i, loss.item(), acc.item())
         )

Epoch: 1/150
Batch number: 000, Training: Loss: 0.8226, Accuracy: 0.3750
Batch number: 001, Training: Loss: 0.6430, Accuracy: 0.6250
Batch number: 002, Training: Loss: 1.6575, Accuracy: 0.6667
Batch number: 003, Training: Loss: 0.8720, Accuracy: 0.6667
Batch number: 004, Training: Loss: 0.5812, Accuracy: 0.5833
Batch number: 005, Training: Loss: 0.3887, Accuracy: 0.8333
Batch number: 006, Training: Loss: 0.5980, Accuracy: 0.7083
Batch number: 007, Training: Loss: 0.2306, Accuracy: 0.8750
Batch number: 008, Training: Loss: 0.1672, Accuracy: 0.9583
Batch number: 009, Training: Loss: 0.3371, Accuracy: 0.8333
Batch number: 010, Training: Loss: 0.1426, Accuracy: 0.9167
Batch number: 011, Training: Loss: 0.7562, Accuracy: 0.7083
Batch number: 012, Training: Loss: 0.0793, Accuracy: 0.9167
Batch number: 013, Training: Loss: 0.2004, Accuracy: 0.9167
Batch number: 014, Training: Loss: 0.1035, Accuracy: 0.9583
Batch number: 015, Training: Loss: 0.2906, Accuracy: 0.8333
Batch number: 016, Training

Batch number: 137, Training: Loss: 0.2470, Accuracy: 0.8750
Batch number: 138, Training: Loss: 0.0760, Accuracy: 1.0000
Batch number: 139, Training: Loss: 0.1654, Accuracy: 0.8750
Batch number: 140, Training: Loss: 0.6106, Accuracy: 0.8333
Batch number: 141, Training: Loss: 0.1976, Accuracy: 0.9583
Batch number: 142, Training: Loss: 0.1157, Accuracy: 0.9583
Batch number: 143, Training: Loss: 0.1603, Accuracy: 0.9583
Batch number: 144, Training: Loss: 0.0339, Accuracy: 1.0000
Batch number: 145, Training: Loss: 0.3467, Accuracy: 0.8750
Batch number: 146, Training: Loss: 0.1524, Accuracy: 0.9167
Batch number: 147, Training: Loss: 0.5737, Accuracy: 0.7500
Batch number: 148, Training: Loss: 0.1032, Accuracy: 1.0000
Batch number: 149, Training: Loss: 0.4635, Accuracy: 0.7083
Batch number: 150, Training: Loss: 0.1396, Accuracy: 0.9583
Batch number: 151, Training: Loss: 0.1307, Accuracy: 0.9583
Batch number: 152, Training: Loss: 0.2681, Accuracy: 0.8750
Batch number: 153, Training: Loss: 0.244

Batch number: 082, Training: Loss: 0.1309, Accuracy: 0.8750
Batch number: 083, Training: Loss: 0.1201, Accuracy: 0.9167
Batch number: 084, Training: Loss: 0.1510, Accuracy: 0.9167
Batch number: 085, Training: Loss: 0.1219, Accuracy: 0.9583
Batch number: 086, Training: Loss: 0.0982, Accuracy: 0.9583
Batch number: 087, Training: Loss: 0.2859, Accuracy: 0.9167
Batch number: 088, Training: Loss: 0.1206, Accuracy: 0.9583
Batch number: 089, Training: Loss: 0.2261, Accuracy: 0.8750
Batch number: 090, Training: Loss: 0.1374, Accuracy: 0.9167
Batch number: 091, Training: Loss: 0.2164, Accuracy: 0.8750
Batch number: 092, Training: Loss: 0.1732, Accuracy: 0.9167
Batch number: 093, Training: Loss: 0.0937, Accuracy: 0.9583
Batch number: 094, Training: Loss: 0.1335, Accuracy: 0.9167
Batch number: 095, Training: Loss: 0.2881, Accuracy: 0.9167
Batch number: 096, Training: Loss: 0.0428, Accuracy: 1.0000
Batch number: 097, Training: Loss: 0.1233, Accuracy: 0.9583
Batch number: 098, Training: Loss: 0.078

Batch number: 027, Training: Loss: 0.0610, Accuracy: 1.0000
Batch number: 028, Training: Loss: 0.1578, Accuracy: 0.9167
Batch number: 029, Training: Loss: 0.1187, Accuracy: 0.9167
Batch number: 030, Training: Loss: 0.1841, Accuracy: 0.9583
Batch number: 031, Training: Loss: 0.1286, Accuracy: 1.0000
Batch number: 032, Training: Loss: 0.6899, Accuracy: 0.7500
Batch number: 033, Training: Loss: 0.2284, Accuracy: 0.9583
Batch number: 034, Training: Loss: 0.1038, Accuracy: 0.9583
Batch number: 035, Training: Loss: 0.0885, Accuracy: 0.9583
Batch number: 036, Training: Loss: 0.4162, Accuracy: 0.8333
Batch number: 037, Training: Loss: 0.1079, Accuracy: 0.9167
Batch number: 038, Training: Loss: 0.1662, Accuracy: 0.8750
Batch number: 039, Training: Loss: 0.5673, Accuracy: 0.6667
Batch number: 040, Training: Loss: 0.0991, Accuracy: 0.9583
Batch number: 041, Training: Loss: 0.0810, Accuracy: 0.9583
Batch number: 042, Training: Loss: 0.2164, Accuracy: 0.8750
Batch number: 043, Training: Loss: 0.044

Batch number: 164, Training: Loss: 0.1384, Accuracy: 0.9583
Batch number: 165, Training: Loss: 0.2048, Accuracy: 0.9583
Batch number: 166, Training: Loss: 0.0819, Accuracy: 0.9583
Batch number: 167, Training: Loss: 0.3174, Accuracy: 0.8750
Batch number: 168, Training: Loss: 0.1420, Accuracy: 0.9167
Batch number: 169, Training: Loss: 0.1685, Accuracy: 0.9167
Batch number: 170, Training: Loss: 0.1719, Accuracy: 0.9583
Batch number: 171, Training: Loss: 0.0921, Accuracy: 1.0000
Batch number: 172, Training: Loss: 0.0601, Accuracy: 0.9583
Batch number: 173, Training: Loss: 0.0307, Accuracy: 1.0000
Batch number: 174, Training: Loss: 0.1371, Accuracy: 0.9583
Batch number: 175, Training: Loss: 0.0533, Accuracy: 1.0000
Batch number: 176, Training: Loss: 0.0295, Accuracy: 1.0000
Batch number: 177, Training: Loss: 0.0550, Accuracy: 0.9583
Batch number: 178, Training: Loss: 0.0582, Accuracy: 1.0000
Batch number: 179, Training: Loss: 0.0496, Accuracy: 1.0000
Batch number: 180, Training: Loss: 0.172

...

Batch number: 190, Training: Loss: 0.1336, Accuracy: 0.9167
Batch number: 191, Training: Loss: 0.2769, Accuracy: 0.9474
Epoch: 11/150
Batch number: 000, Training: Loss: 0.0895, Accuracy: 0.9583
Batch number: 001, Training: Loss: 0.1244, Accuracy: 0.9583
Batch number: 002, Training: Loss: 0.1199, Accuracy: 0.8750
Batch number: 003, Training: Loss: 0.1038, Accuracy: 0.9167
Batch number: 004, Training: Loss: 0.0576, Accuracy: 1.0000
Batch number: 005, Training: Loss: 0.1376, Accuracy: 0.9583
Batch number: 006, Training: Loss: 0.1335, Accuracy: 0.9167
Batch number: 007, Training: Loss: 0.1864, Accuracy: 0.9167
Batch number: 008, Training: Loss: 0.0681, Accuracy: 0.9167
Batch number: 009, Training: Loss: 0.0892, Accuracy: 0.9583
Batch number: 010, Training: Loss: 0.1667, Accuracy: 0.9167
Batch number: 011, Training: Loss: 0.0243, Accuracy: 1.0000
Batch number: 012, Training: Loss: 0.3204, Accuracy: 0.8333
Batch number: 013, Training: Loss: 0.6643, Accuracy: 0.8333
Batch number: 014, Trainin

Batch number: 135, Training: Loss: 0.0295, Accuracy: 1.0000
Batch number: 136, Training: Loss: 0.0612, Accuracy: 1.0000
Batch number: 137, Training: Loss: 0.0875, Accuracy: 0.9583
Batch number: 138, Training: Loss: 0.0852, Accuracy: 1.0000
Batch number: 139, Training: Loss: 0.1261, Accuracy: 0.9167
Batch number: 140, Training: Loss: 0.1160, Accuracy: 0.9583
Batch number: 141, Training: Loss: 0.0636, Accuracy: 0.9583
Batch number: 142, Training: Loss: 0.0352, Accuracy: 1.0000
Batch number: 143, Training: Loss: 0.2967, Accuracy: 0.8750
Batch number: 144, Training: Loss: 0.0579, Accuracy: 1.0000
Batch number: 145, Training: Loss: 0.0785, Accuracy: 0.9583
Batch number: 146, Training: Loss: 0.0715, Accuracy: 1.0000
Batch number: 147, Training: Loss: 0.0227, Accuracy: 1.0000
Batch number: 148, Training: Loss: 0.2048, Accuracy: 0.9167
Batch number: 149, Training: Loss: 0.0821, Accuracy: 0.9583
Batch number: 150, Training: Loss: 0.0531, Accuracy: 0.9583
Batch number: 151, Training: Loss: 0.040

Batch number: 080, Training: Loss: 0.1557, Accuracy: 0.9583
Batch number: 081, Training: Loss: 0.2591, Accuracy: 0.9167
Batch number: 082, Training: Loss: 0.2868, Accuracy: 0.8750
Batch number: 083, Training: Loss: 0.1651, Accuracy: 0.9167
Batch number: 084, Training: Loss: 0.1013, Accuracy: 0.9583
Batch number: 085, Training: Loss: 0.0606, Accuracy: 0.9583
Batch number: 086, Training: Loss: 0.0580, Accuracy: 1.0000
Batch number: 087, Training: Loss: 0.1135, Accuracy: 0.9167
Batch number: 088, Training: Loss: 0.0753, Accuracy: 0.9583
Batch number: 089, Training: Loss: 0.2304, Accuracy: 0.8750
Batch number: 090, Training: Loss: 0.0115, Accuracy: 1.0000
Batch number: 091, Training: Loss: 0.1843, Accuracy: 0.9167
Batch number: 092, Training: Loss: 0.1852, Accuracy: 0.9167
Batch number: 093, Training: Loss: 0.0647, Accuracy: 0.9583
Batch number: 094, Training: Loss: 0.0306, Accuracy: 1.0000
Batch number: 095, Training: Loss: 0.4230, Accuracy: 0.8333
Batch number: 096, Training: Loss: 0.156

Batch number: 025, Training: Loss: 0.1559, Accuracy: 0.9583
Batch number: 026, Training: Loss: 0.0701, Accuracy: 0.9583
Batch number: 027, Training: Loss: 0.0744, Accuracy: 1.0000
Batch number: 028, Training: Loss: 0.1023, Accuracy: 0.9583
Batch number: 029, Training: Loss: 0.2203, Accuracy: 0.9167
Batch number: 030, Training: Loss: 0.2240, Accuracy: 0.8750
Batch number: 031, Training: Loss: 0.2327, Accuracy: 0.9167
Batch number: 032, Training: Loss: 0.2079, Accuracy: 0.9583
Batch number: 033, Training: Loss: 0.1291, Accuracy: 0.9583
Batch number: 034, Training: Loss: 0.1522, Accuracy: 0.9583
Batch number: 035, Training: Loss: 0.1393, Accuracy: 0.9583
Batch number: 036, Training: Loss: 0.1371, Accuracy: 0.9583
Batch number: 037, Training: Loss: 0.1027, Accuracy: 0.9583
Batch number: 038, Training: Loss: 0.1509, Accuracy: 0.9167
Batch number: 039, Training: Loss: 0.0364, Accuracy: 1.0000
Batch number: 040, Training: Loss: 0.2479, Accuracy: 0.8750
Batch number: 041, Training: Loss: 0.158

Batch number: 162, Training: Loss: 0.0816, Accuracy: 0.9583
Batch number: 163, Training: Loss: 0.1181, Accuracy: 0.9167
Batch number: 164, Training: Loss: 0.1109, Accuracy: 0.9167
Batch number: 165, Training: Loss: 0.3153, Accuracy: 0.8750
Batch number: 166, Training: Loss: 0.0550, Accuracy: 1.0000
Batch number: 167, Training: Loss: 0.0710, Accuracy: 1.0000
Batch number: 168, Training: Loss: 0.0562, Accuracy: 0.9583
Batch number: 169, Training: Loss: 0.1685, Accuracy: 0.9167
Batch number: 170, Training: Loss: 0.0841, Accuracy: 0.9583
Batch number: 171, Training: Loss: 0.1692, Accuracy: 0.9167
Batch number: 172, Training: Loss: 0.0700, Accuracy: 0.9583
Batch number: 173, Training: Loss: 0.2958, Accuracy: 0.8333
Batch number: 174, Training: Loss: 0.1587, Accuracy: 0.9583
Batch number: 175, Training: Loss: 0.1288, Accuracy: 0.9167
Batch number: 176, Training: Loss: 0.1845, Accuracy: 0.9167
Batch number: 177, Training: Loss: 0.1192, Accuracy: 0.9583
Batch number: 178, Training: Loss: 0.034

In [ ]:
steps = 287
test_params = {
    'batch_size': 16,
    'shuffle': False,
}

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15, 7)
plt.plot(list(range(len(losses))), losses, label="loss")
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (15, 7)
l = [np.mean(losses[i*steps:(i+1)*steps]) for i in range(epochs)]
len(l)


In [ ]:
plt.plot(list(range(len(l))), l, label="l")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15, 7)
plt.plot(list(range(len(losses))), accs, label="accs")
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (15, 7)
a = [np.mean(accs[i*steps:(i+1)*steps]) for i in range(epochs)]
len(a)
plt.plot(list(range(len(a))), a, label="a")
plt.show()

In [ ]:
submission = pd.read_csv('./hw4/sample_submission.csv')
submission.head()

In [ ]:
X_test = np.array(submission.loc[:, 'image_id'])
Y_test = np.zeros(X_test.shape[0])

In [ ]:
from torch.nn import functional as F

In [ ]:
ds_trans_test = transforms.Compose([
#     torchvision.transforms.Scale(256),
                    torchvision.transforms.CenterCrop(224),
                    torchvision.transforms.ToTensor()])


In [ ]:
import time
test_set = MyDataset(X_test, Y_test, ds_trans_test, datadir='./hw4/test_kaggle')
test_generator = DataLoader(test_set, **test_params)
res = []
for i, (inputs, tt) in enumerate(test_generator):
    inputs = inputs.cuda()
    outputs = resnet(inputs)
    res.append(F.softmax(outputs, dim=1).cpu().data.numpy().argmax(axis=1))
   

In [ ]:
# resnet

In [ ]:
res = np.concatenate(res)
# res

In [ ]:
np.sum(res) / res.shape[0]

In [ ]:
submission['label'] = res
submission.head(20)

In [ ]:
submission.to_csv('./hw4/submission.csv', index=False)